# Lab 3: Deploy Memory-Enhanced Investment Agents to Runtime

This notebook deploys the memory-enhanced 2-agent investment research system from Lab 2 to Amazon Bedrock AgentCore Runtime for production scalability.

## Overview

Building on Lab 2, we'll deploy:
- **Quantitative Analysis Agent**: Runtime deployment (stateless)
- **Memory-Enhanced Supervisor**: Runtime deployment with memory hooks
- **Dynamic ARN Management**: SSM Parameter Store for agent ARNs
- **Cross-Session Memory**: Persistent memory in runtime environment

This follows the **Lab 3** pattern from 07-AgentCore-E2E: deploy local agents with memory to production runtime.

## Prerequisites

- **Lab 2 completed**: Memory-enhanced agents created and tested locally
- Python 3.10+
- Docker or Finch running
- AWS credentials configured
- Amazon Bedrock AgentCore access
- Required Python packages (see requirements.txt)

## Step 1: Environment Setup and Runtime Utilities

Set up the environment with AWS credentials and runtime deployment utilities.

**Important**: Lab 3 requires Docker or Finch to build container images. This lab must be run on:
- Local machine with Docker Desktop installed
- EC2 instance with Docker
- Cloud9 environment
- SageMaker Studio (if Docker/Finch is available)

**Cannot run in**: Standard Jupyter/SageMaker notebooks without Docker

In [ ]:
# Install required packages from requirements.txt
!pip install -q -r requirements.txt bedrock-agentcore-starter-toolkit

In [ ]:
# Import required libraries
import os
import boto3
import json
import time
from datetime import datetime
from IPython.display import display, Markdown

# Runtime deployment imports
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

# AgentCore Memory imports
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType
# Note: HookProvider and events may not be available in current version
# We'll implement memory functionality without hooks for now

print("✅ All libraries imported successfully")

In [ ]:
# Set AWS credentials - Update these with your temporary credentials
# NOTE: Only needed when running locally. Skip this cell if running in SageMaker/Cloud9
# where credentials are automatically provided by the execution role
os.environ['AWS_ACCESS_KEY_ID'] = 'AWS_ACCESS_KEY_ID'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'AWS_SECRET_ACCESS_KEY'
os.environ['AWS_SESSION_TOKEN'] = 'AWS_SESSION_TOKEN'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

print("AWS credentials set successfully!")

# Clear boto3 cached sessions
boto3.DEFAULT_SESSION = None

# Verify credentials and extract account ID
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    AWS_ACCOUNT_ID = identity['Account']
    AWS_REGION = os.environ['AWS_DEFAULT_REGION']
    
    print(f"✅ Using AWS Account: {AWS_ACCOUNT_ID}")
    print(f"✅ User/Role: {identity['Arn']}")
    print(f"✅ Region: {AWS_REGION}")
except Exception as e:
    print(f"❌ Credential verification failed: {e}")
    raise

In [ ]:
# Runtime deployment utilities (from agentcore_runtime_ivr.ipynb)
# These functions handle the deployment of agents to AgentCore Runtime

def get_agentcore_deployment_status(runtime):
    """
    Monitor AgentCore deployment status by polling every 10 seconds.
    
    The deployment goes through several states:
    - CREATING: Building Docker image and pushing to ECR
    - UPDATING: Updating existing deployment
    - READY: Successfully deployed and ready to use
    - CREATE_FAILED/UPDATE_FAILED: Deployment failed
    
    Returns the final status once deployment completes.
    """
    status = 'NOT STARTED'
    status_list = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

    while status not in status_list:
        time.sleep(10)  # Poll every 10 seconds
        status_response = runtime.status()
        status = status_response.endpoint['status']
        print(f"Status: {status}")
    return status

def deploy_agentcore_runtime(agent_name, entry_point, auto_create_execution_role=True, auto_create_ecr=True):
    """
    Deploy agent to AgentCore Runtime.
    
    This function:
    1. Configures the runtime with entry point and requirements
    2. Builds a Docker container image with your agent code
    3. Pushes the image to Amazon ECR (Elastic Container Registry)
    4. Deploys the container to AgentCore Runtime
    5. Returns the agent's ARN for future invocations
    
    Parameters:
    - agent_name: Unique name for the runtime agent
    - entry_point: Path to the Python file with @app.entrypoint decorator
    - auto_create_execution_role: Automatically create IAM role for the agent
    - auto_create_ecr: Automatically create ECR repository for container images
    
    Returns:
    - Agent ARN if successful, or error status if failed
    """
    runtime = Runtime()
    
    # Configure runtime: specify entry point, requirements, and auto-creation options
    conf_response = runtime.configure(
        entrypoint=entry_point,  # Python file with agent code
        auto_create_execution_role=auto_create_execution_role,  # Create IAM role automatically
        auto_create_ecr=auto_create_ecr,  # Create ECR repository automatically
        requirements_file="requirements.txt",  # Python dependencies
        agent_name=agent_name  # Unique agent identifier
    )

    runtime_arn = ''
    # Launch deployment: builds Docker image, pushes to ECR, deploys to runtime
    # auto_update_on_conflict=True allows updating existing deployments
    launch_response = runtime.launch(auto_update_on_conflict=True)
    
    # Monitor deployment status until complete
    launch_status = get_agentcore_deployment_status(runtime)

    if launch_status == 'READY':
        runtime_arn = launch_response.agent_arn
        print(f"✅ Agent deployed successfully: {runtime_arn}")
    else:
        print(f"❌ AgentCore Runtime deployment failed: {launch_status}")
        runtime_arn = launch_status

    return runtime_arn



# Helper function to add SSM permissions
def add_ssm_permissions_to_role(role_name: str, region: str = "us-west-2"):
    """
    Add SSM GetParameter permissions to the runtime execution role.
    This allows the agent to read SSM parameters for memory_id and agent ARNs.
    """
    import boto3
    import json
    
    iam = boto3.client('iam')
    
    # Get account ID
    sts = boto3.client('sts')
    account_id = sts.get_caller_identity()['Account']
    
    policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "ssm:GetParameter",
                    "ssm:GetParameters"
                ],
                "Resource": f"arn:aws:ssm:{region}:{account_id}:parameter/app/investment/agentcore/*"
            }
        ]
    }
    
    try:
        iam.put_role_policy(
            RoleName=role_name,
            PolicyName="AgentCoreSSMAccess",
            PolicyDocument=json.dumps(policy_document)
        )
        print(f"✅ Added SSM permissions to role: {role_name}")
        return True
    except Exception as e:
        print(f"⚠️ Could not add SSM permissions: {e}")
        return False


def get_execution_role_from_arn(agent_arn: str):
    """Extract execution role name from agent ARN by checking the agent configuration."""
    import boto3
    
    # The execution role is stored in the .bedrock_agentcore.yaml
    # For now, we'll use a pattern-based approach
    # Role name format: AmazonBedrockAgentCoreSDKRuntime-{region}-{hash}
    
    # Try to get it from the config file
    try:
        import yaml
        with open('.bedrock_agentcore.yaml', 'r') as f:
            config = yaml.safe_load(f)
            
        # Find the agent in config
        for agent_name, agent_config in config.get('agents', {}).items():
            if agent_config.get('bedrock_agentcore', {}).get('agent_arn') == agent_arn:
                execution_role_arn = agent_config.get('aws', {}).get('execution_role')
                if execution_role_arn:
                    # Extract role name from ARN
                    role_name = execution_role_arn.split('/')[-1]
                    return role_name
    except Exception as e:
        print(f"⚠️ Could not extract role from config: {e}")
    
    return None


def ensure_runtime_has_ssm_access(agent_arn: str, region: str = "us-west-2"):
    """
    Ensure the runtime agent has SSM access.
    Call this after deploying an agent.
    """
    role_name = get_execution_role_from_arn(agent_arn)
    
    if role_name:
        print(f"📋 Found execution role: {role_name}")
        return add_ssm_permissions_to_role(role_name, region)
    else:
        print("⚠️ Could not determine execution role. You may need to add SSM permissions manually.")
        return False


print("✅ Runtime deployment utilities defined")

## Step 2: Setup Memory Resources (from Lab 2)

Retrieve or create the memory resources needed for the runtime agents.

In [ ]:
# Import utilities from Lab 2
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter

# Initialize memory client
memory_client = MemoryClient(region_name=AWS_REGION)

# Get or create memory resource (same as Lab 2)
def create_or_get_memory_resource():
    """Create new memory resource or retrieve existing one using SSM Parameter Store."""
    memory_param_name = "/app/investment/agentcore/memory_id"
    
    try:
        # Try to get existing memory_id from SSM
        memory_id = get_ssm_parameter(memory_param_name)
        print(f"✅ Found existing memory resource: {memory_id}")
        
        # Memory resource found and ready to use
        return memory_id
            
    except ValueError:
        print("❌ No memory found from Lab 2. Please run Lab 2 first.")
        raise ValueError("Memory resource from Lab 2 is required for Lab 3")

# Get memory resource from Lab 2
MEMORY_ID = create_or_get_memory_resource()
print(f"\n🧠 Memory Resource Ready: {MEMORY_ID}")

## Step 2.5: Optional IAM Permissions Setup

**Note**: These cells are optional and only needed if you encounter permission errors.

If running in SageMaker/Jupyter and you get SSM or Bedrock access denied errors, run these cells to add the necessary permissions to your execution role.

In [ ]:
# OPTIONAL: Add SSM permissions to SageMaker/execution role
# Only run this if you get SSM GetParameter/PutParameter access denied errors

import boto3
import json
import time

# Get current role and account info
sts = boto3.client('sts')
identity = sts.get_caller_identity()
role_arn = identity['Arn']
role_name = role_arn.split('/')[-2]  # Extract role name
account_id = identity['Account']
region = boto3.Session().region_name or 'us-west-2'

print(f"Account ID: {account_id}")
print(f"Region: {region}")
print(f"Execution Role: {role_name}")

# Add SSM permissions
iam = boto3.client('iam')

ssm_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ssm:GetParameter",
                "ssm:GetParameters",
                "ssm:PutParameter"
            ],
            "Resource": f"arn:aws:ssm:{region}:{account_id}:parameter/app/investment/agentcore/*"
        }
    ]
}

try:
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="InvestmentAgentSSMAccess",
        PolicyDocument=json.dumps(ssm_policy)
    )
    print(f"✅ Added SSM permissions to {role_name}")
    print(f"   Resource: arn:aws:ssm:{region}:{account_id}:parameter/app/investment/agentcore/*")
    
    # Wait for IAM to propagate
    print("\n⏳ Waiting 15 seconds for IAM permissions to propagate...")
    for i in range(15, 0, -3):
        print(f"   {i} seconds remaining...")
        time.sleep(3)
    print("✅ Ready to proceed!")
    
except Exception as e:
    print(f"❌ Error adding permissions: {e}")
    print("\nYou may need to add these permissions manually in the IAM Console.")

In [ ]:
# OPTIONAL: Add Bedrock model access permissions
# Only run this if you get Bedrock InvokeModel access denied errors

import boto3
import json
import time

# Get current role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
role_name = identity['Arn'].split('/')[-2]

print(f"Adding Bedrock permissions to: {role_name}")

# Add Bedrock permissions
iam = boto3.client('iam')

bedrock_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                "arn:aws:bedrock:*::foundation-model/us.amazon.nova-pro-v1:0",
                "arn:aws:bedrock:*::foundation-model/*"
            ]
        }
    ]
}

try:
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="BedrockModelAccess",
        PolicyDocument=json.dumps(bedrock_policy)
    )
    print("✅ Added Bedrock model access permissions")
    
    print("\n⏳ Waiting 10 seconds for permissions to propagate...")
    time.sleep(10)
    print("✅ Ready!")
    print("\n⚠️ Note: After adding permissions, restart the kernel for changes to take effect.")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nYou may need to add these permissions manually in the IAM Console.")

## Step 3: Create QA Agent for Runtime

Create the Quantitative Analysis Agent for runtime deployment (stateless, no memory).

In [ ]:
%%writefile lab_helpers/lab3_quantitative_agent_runtime.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Import the QA agent from Lab 1 (no changes needed between Lab 1 and Lab 2)
# Note: Since this file is in lab_helpers/, we import without the lab_helpers prefix
from lab_helpers.lab1_quantitative_agent import create_quantitative_analysis_agent

# Initialize AgentCore App
app = BedrockAgentCoreApp()

@app.entrypoint
def qa_strands_agent_bedrock(payload):
    """
    Runtime entrypoint for Quantitative Analysis Agent.
    Reuses Lab 1 agent with runtime wrapper.
    """
    agent = create_quantitative_analysis_agent()
    user_input = payload.get("prompt")
    response = agent(user_input)
    return str(response)

if __name__ == "__main__":
    app.run()

print("✅ QA Agent runtime file created (imports from Lab 1)")


## Step 4: Deploy QA Agent to Runtime

Deploy the Quantitative Analysis Agent to AgentCore Runtime and store its ARN.

In [ ]:
# Deploy QA Agent to runtime
qa_agent_name = "quantitative_analysis_agent_core"
print(f"🚀 Deploying QA Agent: {qa_agent_name}")
print("This may take several minutes...")

qa_agent_arn = deploy_agentcore_runtime(qa_agent_name, "lab_helpers/lab3_quantitative_agent_runtime.py")

if qa_agent_arn.startswith('arn:'):
    # Store ARN in SSM for dynamic retrieval
    put_ssm_parameter(
        "/app/investment/agentcore/qa_agent_arn", 
        qa_agent_arn,
        "QA Agent Runtime ARN for cross-session access"
    )
    print(f"✅ QA Agent ARN stored in SSM: {qa_agent_arn}")
else:
    print(f"❌ QA Agent deployment failed: {qa_agent_arn}")
    raise RuntimeError(f"QA Agent deployment failed: {qa_agent_arn}")

## Step 5: Create Memory-Enhanced Supervisor for Runtime

Create the supervisor agent with memory hooks that calls the runtime QA agent.

In [ ]:
%%writefile lab_helpers/lab3_supervisor_runtime.py
import os
import boto3
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import tool

# Initialize AgentCore App
app = BedrockAgentCoreApp()

# Utility functions
def get_ssm_parameter(parameter_name: str) -> str:
    """Get parameter value from SSM Parameter Store."""
    ssm = boto3.client('ssm')
    try:
        response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
        return response['Parameter']['Value']
    except Exception as e:
        raise ValueError(f"Parameter {parameter_name} not found: {e}")

def get_agent_arn(agent_type: str) -> str:
    """Get agent ARN from SSM Parameter Store."""
    param_name = f"/app/investment/agentcore/{agent_type}_arn"
    return get_ssm_parameter(param_name)

@tool
def invoke_agentcore_runtime(payload, agent_arn):
    """Invoke AgentCore Runtime agent."""
    agentcore_client = boto3.client('bedrock-agentcore', region_name='us-west-2')
    boto3_response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        qualifier="DEFAULT",
        payload=json.dumps(payload)
    )
    content = []
    if "text/event-stream" in boto3_response.get("contentType", ""):
        for line in boto3_response["response"].iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    line = line[6:]
                    content.append(line)
    else:
        try:
            events = []
            for event in boto3_response.get("response", []):
                events.append(event)
        except Exception as e:
            events = [f"Error reading EventStream: {e}"]
        for e in events:
            content.append(e.decode('utf-8'))
    return "\n".join(content)

@tool
def quantitative_analysis_runtime(query: str) -> str:
    """Delegate quantitative analysis tasks to the runtime QA agent."""
    try:
        agent_arn = get_agent_arn("qa_agent")
        user_query = {"prompt": query}
        response = invoke_agentcore_runtime(user_query, agent_arn)
        return response
    except Exception as e:
        return f"Error in quantitative analysis: {str(e)}"

@app.entrypoint
def investment_supervisor_strands_agent_bedrock(payload):
    """
    Runtime entrypoint for memory-enhanced investment supervisor.
    Creates a new agent with runtime QA tool instead of local tool.
    """
    import uuid
    from bedrock_agentcore.memory import MemoryClient
    from strands import Agent
    from strands.models import BedrockModel
    from strands_tools import think
    from lab_helpers.investment_memory_hooks import InvestmentMemoryHooks
    
    # Get memory_id from SSM
    memory_id = get_ssm_parameter("/app/investment/agentcore/memory_id")
    
    # Create memory hooks (same as Lab 2)
    actor_id = "investor_001"
    session_id = str(uuid.uuid4())
    memory_client = MemoryClient(region_name='us-west-2')
    memory_hooks = InvestmentMemoryHooks(memory_id, memory_client, actor_id, session_id)
    
    # Create agent with RUNTIME quantitative_analysis tool
    # Note: We create a new agent here instead of importing from Lab 2
    # because we need to use quantitative_analysis_runtime tool instead of local tool
    agent = Agent(
        name="memory_enhanced_investment_supervisor_runtime",
        system_prompt="""You are a Memory-Enhanced Investment Research Assistant, a financial research supervisor with learning capabilities.

Your enhanced responsibilities:
1. Analyze user queries and leverage previous interaction context when available
2. Delegate quantitative analysis tasks to the quantitative_analysis_runtime tool (calls deployed runtime QA agent)
3. Synthesize analysis with personalized insights based on learned investor preferences
4. Generate investment recommendations that adapt to investor's risk tolerance and interests
5. Build comprehensive investor profiles over time through interaction learning

Memory-Aware Workflow:
1. Review any provided INVESTOR CONTEXT from previous sessions
2. Use quantitative_analysis_runtime tool for current market data and analysis
3. Combine current analysis with learned preferences for personalized recommendations
4. Provide investment advice that considers the investor's historical interests and risk profile

Personalization Guidelines:
- Reference previous stock interests when relevant
- Adapt analysis depth to investor's demonstrated expertise level
- Consider previously expressed risk tolerance in recommendations
- Build on previous investment themes and sector preferences
- Acknowledge learning progression: "Based on your previous interest in..."

Always provide comprehensive, personalized investment analysis that demonstrates learning from previous interactions while incorporating current quantitative data.""",
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0", region_name="us-west-2"),
        tools=[quantitative_analysis_runtime, think],
        hooks=[memory_hooks]
    )

    # Invoke the agent
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message["content"][0]["text"]

if __name__ == "__main__":
    app.run()

print("✅ Memory-Enhanced Supervisor runtime file created (creates new agent with runtime tool)")


## Step 6: Deploy Memory-Enhanced Supervisor to Runtime

Deploy the supervisor agent with memory hooks to AgentCore Runtime.

In [ ]:
# Deploy Memory-Enhanced Supervisor to runtime
supervisor_agent_name = "investment_research_supervisor_runtime_memory"
print(f"🚀 Deploying Memory-Enhanced Supervisor: {supervisor_agent_name}")
print("This may take several minutes...")

supervisor_agent_arn = deploy_agentcore_runtime(supervisor_agent_name, "lab_helpers/lab3_supervisor_runtime.py")

if supervisor_agent_arn.startswith('arn:'):
    # Store ARN in SSM for future reference
    put_ssm_parameter(
        "/app/investment/agentcore/supervisor_agent_arn", 
        supervisor_agent_arn,
        "Memory-Enhanced Supervisor Runtime ARN"
    )
    print(f"✅ Supervisor Agent ARN stored in SSM: {supervisor_agent_arn}")

    # Add SSM permissions to execution role
    print("\n📋 Adding SSM permissions to execution role...")
    ensure_runtime_has_ssm_access(supervisor_agent_arn, AWS_REGION)
else:
    print(f"❌ Supervisor Agent deployment failed: {supervisor_agent_arn}")
    raise RuntimeError(f"Supervisor Agent deployment failed: {supervisor_agent_arn}")

## Step 7: Test Runtime System with Memory

Test the deployed runtime system to ensure memory functionality is preserved.

In [ ]:
# Test runtime system invocation
def invoke_runtime_agent(agent_arn: str, prompt: str, timeout: int = 300):
    """Invoke runtime agent for testing with configurable timeout."""
    from botocore.config import Config
    
    # Configure client with longer timeout for complex queries
    config = Config(
        read_timeout=timeout,
        connect_timeout=60,
        retries={'max_attempts': 3}
    )
    
    agentcore_client = boto3.client(
        'bedrock-agentcore', 
        region_name=AWS_REGION,
        config=config
    )
    
    response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        qualifier="DEFAULT",
        payload=json.dumps({"prompt": prompt})
    )
    
    content = []
    for event in response.get("response", []):
        content.append(event.decode('utf-8'))
    
    # Join and parse the response
    response_text = "\n".join(content)
    
    # Try to parse as JSON if it's wrapped in quotes
    try:
        # If response is a JSON string, parse it
        parsed = json.loads(response_text)
        if isinstance(parsed, str):
            return parsed
        return response_text
    except:
        # If not JSON, return as is
        return response_text

print("🧪 Testing Runtime System...")
print(f"Supervisor ARN: {supervisor_agent_arn}")
print(f"QA Agent ARN: {qa_agent_arn}")
print(f"Memory ID: {MEMORY_ID}")
print("\n⏱️  Note: Complex queries may take 30-60 seconds to complete...")


In [ ]:
# Runtime Session 1: Initial investment analysis with memory
print("🧪 Runtime Session 1: Initial Investment Analysis\n")

session_1_query = """I'm interested in analyzing Tesla (TSLA) stock. 
I'm generally a moderate risk investor focused on long-term growth. 
Please provide a comprehensive analysis."""

try:
    response_1 = invoke_runtime_agent(supervisor_agent_arn, session_1_query)
    print("📊 Runtime Session 1 Response:")
    display(Markdown(response_1))
    print("\n💾 This interaction has been stored in memory for future personalization.")
except Exception as e:
    print(f"❌ Error in runtime session 1: {e}")

In [ ]:
# Runtime Session 2: Follow-up with memory retrieval
print("🧪 Runtime Session 2: Follow-up Analysis (Testing Memory Retrieval)\n")

session_2_query = "How is TSLA performing now compared to when we last discussed it?"

try:
    response_2 = invoke_runtime_agent(supervisor_agent_arn, session_2_query)
    print("📈 Runtime Session 2 Response:")
    display(Markdown(response_2))
    print("\n🧠 Notice how the agent references previous context and provides comparative analysis.")
except Exception as e:
    print(f"❌ Error in runtime session 2: {e}")

In [ ]:
# Runtime Session 3: Personalized recommendations
print("🧪 Runtime Session 3: Personalized Recommendations (Testing Learned Preferences)\n")

session_3_query = "Based on my investment profile, can you recommend some other stocks I might be interested in?"

try:
    response_3 = invoke_runtime_agent(supervisor_agent_arn, session_3_query)
    print("🎯 Runtime Session 3 Response:")
    display(Markdown(response_3))
    print("\n✨ Notice how recommendations are tailored to your previously expressed preferences and risk tolerance.")
except Exception as e:
    print(f"❌ Error in runtime session 3: {e}")

## Step 8: Validation and Summary

Validate the runtime deployment and summarize Lab 3 accomplishments.

In [ ]:
# Runtime deployment validation
print("📋 Lab 3 Runtime Deployment Validation")
print("=" * 50)

# Check QA Agent deployment
try:
    qa_arn_from_ssm = get_ssm_parameter("/app/investment/agentcore/qa_agent_arn")
    qa_deployed = qa_arn_from_ssm == qa_agent_arn
except Exception as e:
    qa_deployed = False

# Check Supervisor deployment
try:
    supervisor_arn_from_ssm = get_ssm_parameter("/app/investment/agentcore/supervisor_agent_arn")
    supervisor_deployed = supervisor_arn_from_ssm == supervisor_agent_arn
except Exception as e:
    supervisor_deployed = False

# Check Memory resource
try:
    memory_info = memory_client.get_memory(memory_id=MEMORY_ID)
    memory_accessible = True
except Exception as e:
    memory_accessible = False
    print(f"⚠️ Memory check failed: {e}")

print(f"✅ QA Agent Deployed: {qa_deployed}")
print(f"✅ QA Agent ARN: {qa_agent_arn}")
print(f"✅ Supervisor Deployed: {supervisor_deployed}")
print(f"✅ Supervisor ARN: {supervisor_agent_arn}")
print(f"✅ Memory Resource Accessible: {memory_accessible}")
print(f"✅ Memory ID: {MEMORY_ID}")
print(f"✅ ARN Management via SSM: {qa_deployed and supervisor_deployed}")
print(f"✅ Runtime System Ready: {qa_deployed and supervisor_deployed and memory_accessible}")

print("\n🎉 Lab 3 Complete!")

## Conclusion

🎉 **Congratulations! You have successfully completed Lab 3: Deploy Memory-Enhanced Investment Agents to Runtime**

### What You Accomplished:

1. **Runtime Deployment** with:
   - QA Agent deployed to AgentCore Runtime (stateless)
   - Memory-Enhanced Supervisor deployed to AgentCore Runtime
   - Production-ready scalability and availability

2. **Dynamic ARN Management** with:
   - ARNs stored in SSM Parameter Store for flexibility
   - No hardcoded ARNs in runtime code
   - Cross-session ARN persistence

3. **Memory Functionality Preserved** with:
   - Same USER_PREFERENCE and SEMANTIC strategies from Lab 2
   - Memory hooks working in runtime environment
   - Cross-session learning maintained in production

4. **Agent Coordination in Runtime** with:
   - Supervisor calling QA agent via runtime ARN
   - Memory-aware delegation and synthesis
   - Personalized investment analysis at scale

### Key Features Demonstrated:

- **Production Scalability**: 24/7 availability with auto-scaling
- **Memory Persistence**: Learning continues across runtime sessions
- **Dynamic Configuration**: ARNs managed via SSM, not hardcoded
- **Agent Orchestration**: Multi-agent coordination in runtime

### Architecture Achieved:
```
User → Supervisor Runtime (Memory) → QA Runtime (Stateless)
  ↓           ↓                           ↓
Memory ← Personalized Response ← Stock Analysis
```

### Files Created:
- `quantitative_analysis_agent_runtime.py` - QA agent for runtime
- `investment_research_supervisor_runtime.py` - Memory-enhanced supervisor for runtime

### Next Steps:
Your investment research system is now production-ready! You can:
- Build a frontend application to interact with the supervisor ARN
- Add more specialist agents (news, risk assessment)
- Implement advanced memory strategies
- Add observability and monitoring

The system will continue learning and becoming a more effective investment research partner with each runtime interaction!

### Resources:
- [Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime.html)
- [AgentCore Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)
- [Strands Agents Documentation](https://github.com/strands-agents/sdk-python)